# CMake

* cmake can take "defines" on the command line that can be used to set paths in the makefiles
    * Example:
~~~~~~~~
cmake -DCMAKE_EXE_LINKER_FLAGS_INIT="-L/Users/stephenh/projects/py-ncepbufr/src" ..
~~~~~~~~

* Other controls  
    https://cmake.org/cmake/help/v3.0/manual/cmake-variables.7.html
    
    CMAKE_ARGC  
    CMAKE_ARGV0  
    CMAKE_BINARY_DIR  
    CMAKE_COMMAND  
    ...  


# Git Flow

## When the developer doesn't have push permission to the github repository

### Developer in their local repo
~~~~~~~~
git flow init

git flow feature start <name>

make edits

git flow feature publish <name>
~~~~~~~~

### Developer on GitHub

~~~~~~~~
initiate a pull request from branch featrue/<name> to branch develop
~~~~~~~~

### Owner on GitHub

~~~~~~~~
once feature/<name> is approved, process the pull request
    merge feature/<name> into develop
    delete feature/<name>
~~~~~~~~

### Developer in their local repo

~~~~~~~~
git branch -D feature/<name>

git remote update -p  # -p prunes the remote feature/<name> branch from the local repo

git checkout master
git pull origin master

git checkout develop
git pull origin develop
~~~~~~~~

# Ecbuild

* To enable features, such as OpenMP, you use the ecbuild_add_option macro
    * The user sets an ENABLE\_ environment var (via ecbuild_add_option)
    * And the macro sets HAVE\_ accordingly
    
    * Example: OpenMP
        * User places the following in a CMakeLists.txt file

~~~~~~~~
ecbuild_add_option( FEATURE OMP
                    DEFAULT ON
                    DESCRIPTION "Use OpenMP" )
~~~~~~~~

* This macro will set the HAVE_OMP cmake variable which is then used downstream by other cmake processes

# How to resolve mege conflict with a git lfs file

```
cd ioda-bundle/iodaconv

git checkout develop
git pull

git checkout feature/mycode
git pull

git merge develop

# for each of the lfs files with conflicts
git checkout --theirs path_to_file    # choose develop version
git checkout --ours path_to_file      # choose feature/mycode version
```

# Build and test fv3-bundle on S4, 1/18/22

* When building my own jedi-stack using intel/impi, the formula for loading modules is a bit tricky. Here is what I did to get this accomplished.

```
git lfs install # I may have not needed this, but it did seem to help

module use /data/users/sherbener/projects/jedi-stack/modulefiles/apps

module load license_intal
module load git/2.30.0
module load miniconda/3.8-s4
module load jedi/intel-impi
module load odc/ecmwf-1.4.4
module load jedi-gcc/8.3

cd /data/users/sherbener/projects
git clone https://github.com/jcsda-internal/fv3-bundle
cd fv3-bundle
git clone https://github.com/jcsda-internal/jedi-cmake jedicmake
mkdir build
cd build
ecbuild --toolchain=../jedicmake/cmake/Toolchains/jcsda-S4-Intel.cmake ..
make -j4
```

# Build jedi-stack with debug settings on libraries, 1/31/22

* MPICH
    * Autoconfig
        * `--enable-g=debug`
        * `--disable-fast`
        * `--enable-debug-info`

* hdf5
    * Autoconfig
        * `--enable-build-mode=debug`
        * `--enable-optimization=debug`

* netcdf
    * Autoconfig
    * export CFLAGS, CXXFLAGS, FFLAGS, FCFLAGS
        * "-g -O0"

* pnetcdf
    * Autocoonfig
        * `--enable-debug`

* eccodes
    * cmake
    * export CFLAGS, etc.

* pio
    * cmake
    * export CFLAGS, etc.

* There are a number of packages that have `--build=production` or `--build=release` settings which can be changed to `--build=Debug`.

# fv3-bundle issue with building eckit, fckit, atlas, 2/23/22

* Tried different combinations of running ecbuild using the `BUNDLE_SKIP_<REPO>` controls which controls whether you build the repo in your bundle (BUNDLE_SKIP_ = OFF) or use the installation in modules (BUNDLE_SKIP_ = ON, which is the default).

| eckit | fckit | atlas | ecbuild status |
|-------|-------|-------|----------------|
| bundle | bundle | bundle | Fail |
| bundle | bundle | module | Pass |
| bundle | module | bundle | Fail |
| bundle | module | moduel | Pass |
| module | bundle | bundle | Pass |
| module | bundle | module | Pass |
| module | module | bundle | Pass |
| module | module | moduel | Pass |

* It appears that building eckit and atlas together in the bundle does not work. All other combinations do work.

* Used `--trace-expand` to see what's going on with a failing combination
    * Found that `ENABLE_OMP=OFF` when the atlas `atlas-import.cmake` file is being constructed
    * This causes the `atlas_OMP, atlas_OMP_Fortran, atlas_OMP_CXX` features to be recorded in the `atlas-import.cmake` file as unavailable.
    * Downstream oops is looking for these features, via a check script in eckit, that doesn't find them and throws the error

* Found that adding `-DENABLE_OMP=ON` to the ecbuild command line fixes the problem since `ENABLE_OMP=ON` when the `atlas-import.cmake` file is being constructed.